In [3]:
##############################
### COMPLETE IMPLEMENTATION ###
##############################

import os
import time
import json
import requests
import logging
import pickle
import hashlib
import tempfile
from dataclasses import dataclass, asdict
from typing import List, Dict, Tuple, Optional
from concurrent.futures import ThreadPoolExecutor, TimeoutError as FutureTimeoutError

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# ======================
# CORE DATA STRUCTURES
# ======================
@dataclass
class TherapeuticResponse:
    VERSION = 2
    text: str
    timestamp: float
    error: bool = False
    processing_time: float = 0.0
    error_details: str = ""
    timeout: bool = False
    empathy_score: float = 0.0
    safety_checks: List[str] = None
    ethical_considerations: List[str] = None
    refinement_suggestions: List[str] = None
    crisis_flag: bool = False
    context_hash: str = ""

    def __getstate__(self):
        state = asdict(self)
        state['_version'] = self.VERSION
        return state

    def __setstate__(self, state):
        if state.get('_version', 1) < self.VERSION:
            state.setdefault('context_hash', "")
        self.__dict__.update(state)

# ======================
# UTILITIES
# ======================
class AtomicPickler:
    @staticmethod
    def save(obj, path):
        try:
            with tempfile.NamedTemporaryFile(mode='wb', delete=False, dir=os.path.dirname(path)) as tmp:
                pickle.dump(obj, tmp, protocol=pickle.HIGHEST_PROTOCOL)
                os.replace(tmp.name, path)
        except Exception as e:
            logger.error(f"Atomic save failed: {str(e)}")

    @staticmethod
    def load(path):
        try:
            with open(path, 'rb') as f:
                return pickle.load(f)
        except Exception as e:
            logger.error(f"Loading failed: {str(e)}")
            raise

# ======================
# CORE COMPONENTS
# ======================
class OllamaClient:
    def __init__(self, model_name: str = "hf.co/TheDrummer/Gemmasutra-Mini-2B-v1-GGUF:Q3_K_L", base_url: str = "http://localhost:11434"):
        self.model_name = model_name
        self.base_url = base_url
        self.max_retries = 5
        self.request_timeout = 600  # Increased timeout
        self._session = requests.Session()

    def _parse_json_safe(self, text: str):
        clean_text = text.strip()
        try:
            return json.loads(clean_text)
        except json.JSONDecodeError:
            start = clean_text.find('{')
            end = clean_text.rfind('}') + 1
            return json.loads(clean_text[start:end])
        except:
            return {"error": "Invalid JSON format"}

    def generate(self, prompt: str) -> Tuple[str, bool]:
        for attempt in range(self.max_retries):
            try:
                response = self._session.post(
                    f"{self.base_url}/api/generate",
                    json={
                        "model": self.model_name,
                        "prompt": prompt[:4000],
                        "stream": False,
                        "options": {"temperature": 0.5}
                    },
                    timeout=self.request_timeout
                )
                data = self._parse_json_safe(response.text)
                return data.get("response", ""), False
            except Exception as e:
                logger.warning(f"Attempt {attempt+1} failed: {e}")
                time.sleep(2 ** attempt)
        return f"Error: Failed after {self.max_retries} attempts", True

class BaseAgent:
    def __init__(self, client: OllamaClient):
        self.client = client
        self.retry_count = 3
        self.max_wait = 600  # Increased timeout
        self._state_dir = "agent_states"
        os.makedirs(self._state_dir, exist_ok=True)

    def safe_generate(self, prompt: str) -> TherapeuticResponse:
        start_time = time.time()
        try:
            text, error = self.client.generate(prompt)
            logger.info(f"Generated response: {text[:200]}...")
            return TherapeuticResponse(
                text=text,
                timestamp=start_time,
                error=error,
                processing_time=time.time() - start_time
            )
        except Exception as e:
            return TherapeuticResponse(
                text=f"Error: {str(e)}",
                timestamp=start_time,
                error=True,
                processing_time=time.time() - start_time
            )

class IntimacyActionGenerator(BaseAgent):
    def generate_actions(self, analysis: Dict) -> List[Dict]:
        prompt = f"""Generate intimacy-enhancing actions based on:
        {json.dumps(analysis, indent=2)[:2000]}
        Focus on mutual consent and emotional connection."""
        
        response = self.safe_generate(prompt)
        logger.info(f"Action Generation Response: {response.text[:200]}...")
        return self.client._parse_json_safe(response.text).get('actions', [])

class TherapeuticResponseSystem:
    def __init__(self):
        self.client = OllamaClient()
        self.agents = {
            'generator': IntimacyActionGenerator(self.client),
            'context': BaseAgent(self.client),
            'safety': BaseAgent(self.client)
        }
        self.history = []

    def process_request(self, input_text: str) -> TherapeuticResponse:
        try:
            # Context analysis
            context = self.agents['context'].safe_generate(f"Analyze: {input_text}")
            logger.info(f"Context Analysis: {context.text[:200]}...")
            
            # Action generation
            analysis = self.client._parse_json_safe(context.text)
            actions = self.agents['generator'].generate_actions(analysis)
            
            # Safety check
            safety_check = self.agents['safety'].safe_generate(
                f"Validate safety of: {json.dumps(actions)}"
            )
            logger.info(f"Safety Check: {safety_check.text[:200]}...")
            
            # Build response
            response = TherapeuticResponse(
                text=json.dumps(actions),
                timestamp=time.time(),
                context_hash=hashlib.sha256(input_text.encode()).hexdigest()
            )
            
            # Save outputs
            AtomicPickler.save(response, "last_response.pkl")
            logger.info("Response saved to last_response.pkl")
            
            print(f"\nFinal Response:\n{response.text}")
            return response
            
        except Exception as e:
            logger.error(f"Processing failed: {str(e)}")
            return TherapeuticResponse(
                text=f"System Error: {str(e)}",
                timestamp=time.time(),
                error=True
            )

if __name__ == "__main__":
    system = TherapeuticResponseSystem()
    sample_input = "How can we improve emotional intimacy in our relationship?"
    print(f"Processing sample request: {sample_input}")
    response = system.process_request(sample_input)
    print("\nResponse Object:")
    print(f"- Text: {response.text[:200]}...")
    print(f"Error: {response.error}")
    print(f"Duration: {response.processing_time:.2f}s")


Processing sample request: How can we improve emotional intimacy in our relationship?


2025-02-21 12:08:39,794 - INFO - Generated response: To improve emotional intimacy in your relationship, you and your partner need to share deep feelings, thoughts, and experiences with each other. This requires open communication, vulnerability, and a ...
2025-02-21 12:08:39,799 - INFO - Context Analysis: To improve emotional intimacy in your relationship, you and your partner need to share deep feelings, thoughts, and experiences with each other. This requires open communication, vulnerability, and a ...
2025-02-21 12:08:39,800 - ERROR - Processing failed: Expecting value: line 1 column 1 (char 0)



Response Object:
- Text: System Error: Expecting value: line 1 column 1 (char 0)...
Error: True
Duration: 0.00s
